# Create inputs for a variable-coefficient KdV run from the climatology atlas

Steps:
---

    1. Input a transect and time period
    2. Extract the topography along the line
    3. Extract the buoyancy along the line
    4. Extract a time-series of internal tide amplitude moving onto the shelf
    
    

In [ ]:
from iwatlas import sshdriver
from iwatlas import harmonics
import iwatlas.stratification as strat
from iwatlas import iwaves



import xarray as xr
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d, RectBivariateSpline

import matplotlib.pyplot as plt

In [ ]:
basedir = '/home/suntans/cloudstor/Data/IWAtlas-lite'


# climfile = '{}/NWS_2km_GLORYS_hex_2013_2014_Climatology.nc'.format(basedir)
N2file = '{}/NWS_2km_GLORYS_hex_2013_2014_Stratification_Atlas.nc'.format(basedir)
sshfile = '{}/NWS_2km_GLORYS_hex_2013_2014_SSHBC_Harmonics.nc'.format(basedir)


ssh_ds = sshdriver.load_ssh_clim(sshfile)
N2_ds = sshdriver.load_ssh_clim(N2file)
N2_ds

In [ ]:
######
# Specify a start and end point and draw on a map

## WEL Bathy Prelude transect
x0 = 122.753
y0 = -13.1026

x1 = 123.486
y1 = -13.947

# Prelude point
xpt = 123.3506
ypt = -13.7641

dx = 0.01 # ~1km
######

In [ ]:
## Generate x and y slice coordinates
lon = np.arange(x0, x1, dx)
Fx = interp1d([x0,x1],[y0,y1])
lat = Fx(lon)

In [ ]:
plt.figure(figsize=(8,6))
N2_ds.plotcelldata(N2_ds._ds.dv, vmax=1000, cmap='gist_earth_r')
N2_ds.contourf(N2_ds._ds.dv, [100,200,300,400,500,600], colors='k', linewidths=0.2, filled=False)
plt.plot(lon,lat,'r--')
plt.plot(xpt, ypt, 'm*')
plt.xlim(xpt-2, xpt+2)
plt.ylim(ypt-2, ypt+2)

In [ ]:
# Interpolate the bathy along the line
hbathy = N2_ds.interpolate(N2_ds._ds.dv, lon, lat)

plt.figure()
plt.plot(lon, -hbathy)

In [ ]:
# Interpolate the buoyancy frequency along the line

timept = np.array([
    np.datetime64('2017-03-30 00:00:00'),
    np.datetime64('2017-03-01 00:00:00'),
    np.datetime64('2016-08-01 00:00:00'),

    ])
Nz = 80

N2_z, zout = strat.predict_N2(N2file, lon ,lat, timept,  Nz)

In [ ]:
# Plot the buoyancy frequency at the first point in space
plt.figure()
plt.plot(N2_z[:,0,0], -zout[:,0,0],)
plt.plot(N2_z[:,0,1], -zout[:,0,1],)

In [ ]:
# Calculate the internal wave parameters
phi_n, cn = iwaves.calc_modes(N2_z, zout)
alpha = iwaves.calc_alpha(phi_n, cn, zout)
beta = iwaves.calc_beta(phi_n, cn, zout)
ampratio = iwaves.ssh_to_amp_ratio(N2_z, phi_n, zout)
sshratio = iwaves.amp_to_ssh_ratio(N2_z, phi_n, zout)

In [ ]:
plt.figure(figsize=(6,10))
plt.subplot(411)
plt.plot(lon, -hbathy)
plt.ylim(-600,0)

plt.subplot(412)
plt.plot(lon, cn[:,0])
plt.plot(lon, cn[:,1])

plt.ylim(1,2)

plt.subplot(413)
plt.plot(lon, alpha[:,0])
plt.plot(lon, alpha[:,1])

plt.ylim(-0.012, 0.012)

plt.subplot(414)
plt.plot(lon, beta[:,0])
plt.plot(lon, beta[:,1])
plt.ylim(0,3e4)

In [ ]:
# Extract the slowly-varying amplitudes for one point in space and time

xyrange = 2.5
xlims = (x0-xyrange, x0+xyrange)
ylims = (y0-xyrange, y0+xyrange)



# Filtering bands (degrees CCW from East)
thetalow = 270
thetahigh = 360


# thetalow = 60
# thetahigh = 120
#######

A_re_f, A_im_f, A_re, A_im, X, Y = sshdriver.extract_amp_nonstat_dff(ssh_ds, xlims, ylims, dx, timept,\
                    thetalow, thetahigh, A_re=None, A_im=None)

In [ ]:
frq=0
z = A_re[frq,0,...] + 1j*A_im[frq,0,...]
zf = A_re_f[frq,0,...] + 1j*A_im_f[frq,0,...]

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(121)
plt.pcolormesh(X,Y,np.abs(z), cmap='Reds')
plt.colorbar()
plt.gca().set_aspect('equal')
plt.plot(x0,y0,'kd')

plt.subplot(122)
plt.pcolormesh(X,Y,np.abs(zf), cmap='Reds')
plt.colorbar()
plt.tight_layout()
plt.gca().set_aspect('equal')
plt.plot(x0,y0,'kd')


In [ ]:
scalefac = sshratio.mean() # ~250:1 ssh:amplitude ratio

plt.figure(figsize=(12,6))
plt.subplot(121)
plt.pcolormesh(X,Y,np.real(z)*scalefac, cmap='RdBu')
plt.colorbar()
plt.gca().set_aspect('equal')
N2_ds.contourf(N2_ds._ds.dv, [100,200,300,400,500,600], colors='k', linewidths=0.2, filled=False)
plt.plot(lon,lat,'r--')
plt.plot(xpt, ypt, 'm*')
plt.plot(x0,y0,'kd')
plt.xlim(xlims)
plt.ylim(ylims)

plt.subplot(122)
plt.pcolormesh(X,Y,np.imag(zf)*scalefac, cmap='RdBu')
plt.colorbar()
plt.tight_layout()
plt.gca().set_aspect('equal')
N2_ds.contourf(N2_ds._ds.dv, [100,200,300,400,500,600], colors='k', linewidths=0.2, filled=False)
plt.plot(lon,lat,'r--')
plt.plot(xpt, ypt, 'm*')
plt.plot(x0,y0,'kd')
plt.xlim(xlims)
plt.ylim(ylims)


In [ ]:
# Interpolate the DFF result back onto the point of interest
nf, nt, ny, nx = A_re.shape

A_re_pt = np.zeros((nf,nt))
A_im_pt = np.zeros((nf,nt))
for ff in range(nf):
    for ii in range(nt):
        F = RectBivariateSpline( Y[:,0], X[0,:], A_re_f[ff,ii,...])
        A_re_pt[ff,ii] = F(y0,x0)
        F = RectBivariateSpline( Y[:,0], X[0,:], A_im_f[ff,ii,...])
        A_im_pt[ff,ii] = F(y0,x0)

In [ ]:
# Generate a time-series using amplitudes from the first time steps only
omegaall = ssh_ds._ds.omega.values
na = 3
ntide = 5
# Location of the main tidal frequencies
ii = [(ff+1)*(2*na+1)-na-1 for ff in range(ntide)]
omega = omegaall[ii]
reftime = np.datetime64(ssh_ds._ds.attrs['ReferenceDate'])


dt = 1800
nsteps = 15*86400//dt
timeout = [timept[0]+np.timedelta64(ii*dt,'s') for ii in range(nsteps)]

tsec = (timeout - reftime).astype('timedelta64[s]').astype(float)
omega,ii
ssh_pt_f = harmonics.harmonic_pred(0, A_re_pt[:,0], A_im_pt[:,0], omega, tsec)

In [ ]:
# # Calculate a time series of ssh at the point
ssh_pt = sshdriver.predict_ssh(sshfile, x0, y0, timeout)

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(timeout, ssh_pt*scalefac)
plt.plot(timeout, ssh_pt_f*scalefac)